In [2]:
from pyod.models.knn import KNN   # imprt kNN分类器

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/CC_data/feature.csv")

In [5]:
df.head()

,ip_sum,ip_remote_null,ip_remote_notnull,ip_real_null,ip_real_notnull,ip_forward_null,ip_forward_notnull,ip_remote_real_equal,ip_black_cf,ip_black_waf,...,xss_html_transform_entropy,command_special_char_count,command_special_char_dist,command_special_char_max,command_special_char_min,command_special_char_mean,command_special_char_var,command_special_char_std,command_special_char_entropy,label
0,6,0,6,6,0,0,6,0,0,0,...,1.584963,0.0,1,27.0,27.0,27.0,0.0,0.0,0.0,0
1,6,0,6,6,0,0,6,0,0,0,...,1.584963,0.0,1,27.0,27.0,27.0,0.0,0.0,0.0,0
2,1,0,1,1,0,0,1,0,0,0,...,1.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,6,0,6,6,0,0,6,0,0,0,...,1.584963,0.0,1,27.0,27.0,27.0,0.0,0.0,0.0,0
4,1,0,1,1,0,0,1,0,0,0,...,1.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [6]:
df.label.sum()

103

In [7]:
df.shape

(556769, 194)

In [8]:
x_train = df.drop(['label'],axis=1)

In [9]:
x_train.shape

(556769, 193)

In [10]:
x_train.head()

,ip_sum,ip_remote_null,ip_remote_notnull,ip_real_null,ip_real_notnull,ip_forward_null,ip_forward_notnull,ip_remote_real_equal,ip_black_cf,ip_black_waf,...,xss_html_transform_std,xss_html_transform_entropy,command_special_char_count,command_special_char_dist,command_special_char_max,command_special_char_min,command_special_char_mean,command_special_char_var,command_special_char_std,command_special_char_entropy
0,6,0,6,6,0,0,6,0,0,0,...,14.764823,1.584963,0.0,1,27.0,27.0,27.0,0.0,0.0,0.0
1,6,0,6,6,0,0,6,0,0,0,...,14.764823,1.584963,0.0,1,27.0,27.0,27.0,0.0,0.0,0.0
2,1,0,1,1,0,0,1,0,0,0,...,2.000000,1.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0,6,6,0,0,6,0,0,0,...,14.764823,1.584963,0.0,1,27.0,27.0,27.0,0.0,0.0,0.0
4,1,0,1,1,0,0,1,0,0,0,...,2.500000,1.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 训练一个kNN检测器
clf_name = 'kNN'
clf = KNN() # 初始化检测器clf

clf.fit(x_train)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [12]:
# 返回训练数据X_train上的异常标签和异常分值
y_train = df.label
y_train_pred = clf.labels_  # 返回训练数据上的分类标签 (0: 正常值, 1: 异常值)
y_train_scores = clf.decision_scores_  # 返回训练数据上的异常值 (分值越大越异常)

# 用训练好的clf来预测未知数据中的异常值
# y_test_pred = clf.predict(X_test)  # 返回未知数据上的分类标签 (0: 正常值, 1: 异常值)
# y_test_scores = clf.decision_function(X_test)  #  返回未知数据上的异常值 (分值越大越异常)

In [13]:
# 评估预测结果
from pyod.utils.data import evaluate_print

print("\nOn Train Data:")
evaluate_print(clf_name, y_train, y_train_scores)

# 可视化
# visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
#     y_test_pred, show_figure=True, save_figure=False)


On Train Data:
kNN ROC:0.9557, precision @ rank n:0.0291


In [14]:
# train an ECOD detector
from pyod.models.ecod import ECOD
clf = ECOD()
clf.fit(x_train)

# get outlier scores
y_train_scores = clf.decision_scores_  # raw outlier scores on the train data

In [15]:
# 评估预测结果

print("\nOn Train Data:")
evaluate_print(clf_name, y_train, y_train_scores)



On Train Data:
kNN ROC:0.9821, precision @ rank n:0.5631


In [16]:
y_train_scores

array([101.46113513, 102.45021876,  82.85831271, ...,  77.14343609,
        64.97827069,  73.8166609 ])

In [17]:
y_train_pred = clf.labels_  # 返回训练数据上的分类标签 (0: 正常值, 1: 异常值)

In [18]:
y_train_pred

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
y_train_pred.sum()

55677

In [20]:
df_pred = pd.DataFrame()
df_pred['y_train'] = y_train
df_pred['y_train_pred'] = y_train_pred
df_pred['y_train_scores'] = y_train_scores

In [21]:
df_pred.head()

,y_train,y_train_pred,y_train_scores
0,0,0,101.461135
1,0,0,102.450219
2,0,0,82.858313
3,0,0,101.540032
4,0,0,78.125772


In [22]:
print(df_pred.y_train.sum())
print(df_pred.y_train_pred.sum())

103
55677


In [23]:
df_pred['pred_rank'] = df_pred['y_train_scores'].rank(ascending=False, method='first')

In [24]:
df_pred[df_pred.pred_rank == df_pred.pred_rank.min()]

,y_train,y_train_pred,y_train_scores,pred_rank
345451,1,1,893.718561,1.0


In [25]:
df_pred[df_pred.pred_rank <= 55677]['y_train_pred'].sum()

55677

In [26]:
df_pred[df_pred.pred_rank <= 200]['y_train'].sum()

63

In [27]:
df_pred[df_pred.pred_rank <= 100]['y_train'].sum()

57

In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [29]:
precision = precision_score(y_train, y_train_pred, average='binary')
recall = recall_score(y_train, y_train_pred, average='binary')
f1score = f1_score(y_train, y_train_pred, average='binary')
 
print(precision)
print(recall)
print(f1score)

0.0018140345205381037
0.9805825242718447
0.0036213696665471497


In [30]:
# train IForest detector
from pyod.models.iforest import IForest
clf_name = 'IForest'
clf = IForest()
clf.fit(x_train)

IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0)

In [31]:
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

In [32]:
precision = precision_score(y_train, y_train_pred, average='binary')
recall = recall_score(y_train, y_train_pred, average='binary')
f1score = f1_score(y_train, y_train_pred, average='binary')
 
print(precision)
print(recall)
print(f1score)

0.0018140345205381037
0.9805825242718447
0.0036213696665471497


In [33]:
from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP
from pyod.models.ocsvm import OCSVM

In [34]:
100/500000.00

0.0002

In [35]:
#设定异常值比例
outliers_fraction = 0.0002
 
# 初始化 LSCP 探测集
detector_list = [LOF(n_neighbors=5), LOF(n_neighbors=10), LOF(n_neighbors=15),
                 LOF(n_neighbors=20), LOF(n_neighbors=25), LOF(n_neighbors=30),
                 LOF(n_neighbors=35), LOF(n_neighbors=40), LOF(n_neighbors=45),
                 LOF(n_neighbors=50)]
classifiers = {
    'Angle-based Outlier Detector (ABOD)':
        ABOD(contamination=outliers_fraction),
    'Cluster-based Local Outlier Factor (CBLOF)':
        CBLOF(contamination=outliers_fraction,
              check_estimator=False, random_state=0),
    'Feature Bagging':
        FeatureBagging(LOF(n_neighbors=35),
                       contamination=outliers_fraction,
                       random_state=0),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=0),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers_fraction),
    'Average KNN': KNN(method='mean',
                       contamination=outliers_fraction),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers_fraction),
    'Minimum Covariance Determinant (MCD)': MCD(
        contamination=outliers_fraction, random_state=0),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers_fraction, random_state=0),
    'Locally Selective Combination (LSCP)': LSCP(
        detector_list, contamination=outliers_fraction,
        random_state=0)
}
 
for i, clf in enumerate(classifiers.keys()):
    print('Model', i + 1, clf)

Model 1 Angle-based Outlier Detector (ABOD)
Model 2 Cluster-based Local Outlier Factor (CBLOF)
Model 3 Feature Bagging
Model 4 Histogram-base Outlier Detection (HBOS)
Model 5 Isolation Forest
Model 6 K Nearest Neighbors (KNN)
Model 7 Average KNN
Model 8 Local Outlier Factor (LOF)
Model 9 Minimum Covariance Determinant (MCD)
Model 10 One-class SVM (OCSVM)
Model 11 Principal Component Analysis (PCA)
Model 12 Locally Selective Combination (LSCP)


In [36]:
import matplotlib.pyplot as plt

In [ ]:
for i, (clf_name, clf) in enumerate(classifiers.items()):
    #训练数据
    clf.fit(x_train)
    # 预测异常值分数
    scores_pred = clf.decision_function(x_train) * -1
    # 预测异常值和正常值的数据
    y_pred = clf.predict(x_train)
    y_train_scores = clf.decision_scores_
    
    precision = precision_score(y_train, y_train_pred, average='binary')
    recall = recall_score(y_train, y_train_pred, average='binary')
    f1score = f1_score(y_train, y_train_pred, average='binary')
    
    print(clf_name)
    print(precision)
    print(recall)
    print(f1score)

 

/data1/liutao6/anaconda3/envs/store_risk/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/data1/liutao6/anaconda3/envs/store_risk/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/data1/liutao6/anaconda3/envs/store_risk/lib/python3.10/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/data1/liutao6/anaconda3/envs/store_risk/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/data1/liutao6/anaconda3/envs/store_risk/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encoun

Angle-based Outlier Detector (ABOD)
0.0018140345205381037
0.9805825242718447
0.0036213696665471497
Cluster-based Local Outlier Factor (CBLOF)
0.0018140345205381037
0.9805825242718447
0.0036213696665471497
